In [1]:
from fsspec_reference_maker.hdf import SingleHdf5ToZarr 
from fsspec_reference_maker.combine import MultiZarrToZarr

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import zipfile
import logging
import fsspec
import json
from tqdm import tqdm
from glob import glob
import os

from azure.storage.blob import ContainerClient
import tempfile

In [3]:
from dask.distributed import Client
client = Client()
client

/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39451 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:43237 Dashboard: /proxy/8787/status,Cluster Workers: 4 Cores: 4 Memory: 32.00 GiB


In [ ]:
%%time
fs =  fsspec.filesystem('reference', fo='combined.json', 
                      remote_protocol='abfs', remote_options={'account_name':'goeseuwest'})

In [10]:
m = fs.get_mapper("")

In [12]:
%%time
ds = xr.open_dataset(m, engine='zarr')

CPU times: user 12.6 s, sys: 120 ms, total: 12.8 s
Wall time: 12.8 s


In [26]:
ds.t.resample(t="1H").nearest(tolerance='1H')

<xarray.DataArray 't' (t: 24)>
array(['2020-01-02T00:05:07.062280960', '2020-01-02T00:55:07.052914944',
       '2020-01-02T01:55:07.327035008', '2020-01-02T02:55:07.028939904',
       '2020-01-02T03:55:07.296160000', '2020-01-02T04:55:07.580553088',
       '2020-01-02T05:55:07.846258944', '2020-01-02T06:55:07.801010048',
       '2020-01-02T07:55:07.514807040', '2020-01-02T08:55:07.505992064',
       '2020-01-02T09:55:07.793617920', '2020-01-02T10:55:07.768898048',
       '2020-01-02T11:55:07.815037056', '2020-01-02T12:55:07.797641088',
       '2020-01-02T13:55:07.207019008', '2020-01-02T14:55:07.489636992',
       '2020-01-02T15:55:07.491003008', '2020-01-02T16:55:07.788180992',
       '2020-01-02T17:55:07.566337920', '2020-01-02T18:55:07.268445056',
       '2020-01-02T19:55:07.559703936', '2020-01-02T20:55:06.977600000',
       '2020-01-02T21:55:07.554075008', '2020-01-02T22:55:07.538180992'],
      dtype='datetime64[ns]')
Coordinates:
  * t        (t) datetime64[ns] 2020-01-02 ... 2020-01-02T23:00:00
    x_image  float64 0.0
    y_image  float64 0.0
Attributes:
    axis:           T
    bounds:         time_bounds
    long_name:      J2000 epoch mid-point between the start and end image sca...
    standard_name:  time